# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)
print(len(train_loader))
print(len(test_loader))

468
78


Then, we define the model, object function and optimizer that we use to classify.

In [3]:
import torch.nn.functional as F
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Conv2d(1,16,kernel_size=3,stride=1,padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16,32,kernel_size=3,stride=1,padding=1)
        self.fc1 = nn.Linear(32*7*7,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,10)
        
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*7*7) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))   
        x = self.fc3(x)  
        return x
model = SimpleNet()

import torch.optim as optim
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

Next, we can start to train and evaluate!

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        scores = model(images)
        loss = criterion(scores,labels)
        loss.backward()
        optimizer.step()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    model.eval()
    correct_test = 0
    correct_train = 0
for data_test in test_loader:
    images, labels = data_test
    output_test = model(images)
    _, predicted = torch.max(output_test, 1)
    correct_test += (predicted == labels).sum()
    test_acc = correct_test.item()/len(test_dataset)
    
 
for data_train in train_loader:
    images_train, labels_train = data_train
    output_train = model(images_train)
    _, predicted_train = torch.max(output_train, 1)
    correct_train += (predicted_train == labels_train).sum()
    train_acc = correct_train.item()/len(train_dataset)
    

100%|██████████| 468/468 [00:53<00:00,  8.73it/s]


#### Q5:
Please print the training and testing accuracy.

In [5]:
print('Training accuracy: %0.2f%%' % (train_acc*100))
print('Testing accuracy: %0.2f%%' % (test_acc*100))

Training accuracy: 97.35%
Testing accuracy: 97.39%
